In [1]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
%load_ext google.cloud.bigquery

# Set your default project here
pandas_gbq.context.project = 'bigquery-public-data'
pandas_gbq.context.dialect = 'standard'

# Load Data

The BigQuery Python client library provides a magic command that allows you to run queries with minimal code.

The BigQuery client library provides a cell magic, %%bigquery. The %%bigquery magic runs a SQL query and returns the results as a pandas `DataFrame`

TODO:
- [ ] Slide 1 - Executive Summary
- [ ] Slide 2 - Brooklyn vs rest of NY, why is Brooklyn an issue?
- [ ] Slide 3 - Focus on Brooklyn: main causes, day of accidents, correlation with weather
- [ ] Slide 4 - Recommendations: Dashboard with live updates, campaign against distraction
- [ ] Slide 5 - Technicalities and approach taken to analysis 

## Slide 2 - Brooklyn vs rest of NY, why is Brooklyn an issue?

In [5]:
%%bigquery injured_df
#number of injured per day, by borough
SELECT
  date(timestamp) as date,
  borough,
  SUM(number_of_persons_injured) as tot_injured
FROM
  `bigquery-public-data.new_york.nypd_mv_collisions`
GROUP BY
  borough,
  date(timestamp) 

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12186/12186 [00:02<00:00, 4473.91rows/s]


In [6]:
injured_df

,date,borough,tot_injured
0,2018-01-17,QUEENS,20
1,2015-05-08,MANHATTAN,29
2,2017-11-22,QUEENS,28
3,2017-12-05,BRONX,11
4,2017-11-03,BRONX,27
...,...,...,...
12181,2016-06-05,STATEN ISLAND,2
12182,2015-01-11,STATEN ISLAND,2
12183,2015-02-25,STATEN ISLAND,0
12184,2014-08-23,STATEN ISLAND,6


In [3]:
#number of killed per day, by borough